In [ ]:
import nuclio 
import mlrun

In [ ]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [ ]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames

In [ ]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/ml-models"

In [18]:
# nuclio: start-code

In [19]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd
import os

In [37]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    stock_list = []
    # Reading the data from feature service
    service = fs.get_online_feature_service("stocks-vec")
    for key,value in context.sym_to_url.items():
        context.logger.info(key)
        context.logger.info(service.get([{"symbol": key}],as_list = True))
        stock_list.append(*service.get([{"symbol": key}],as_list = True))
        
    columns = [{'text': key, 'type': 'object'} for key in ["stocks.prices_min_1h",
                "stocks.prices_max_1h",
                "stocks.volumes_min_1h",
                "stocks.volumes_max_1h",
                "stocks.price",
                "stocks.volume",
                "stocks.last_updated",
                "news.sentiments_min_1h",
                "news.sentiments_max_1h",
                "news.sentiment"]]
    
    #data = [list(item.values()) for item in context.data]
    response = [{'columns': columns,
                'rows': stock_list,
                'type': 'table'}]
    return response

In [38]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-test-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.prices_min_1h",
                "stocks.prices_max_1h",
                "stocks.volumes_min_1h",
                "stocks.volumes_max_1h",
                "stocks.price",
                "stocks.volume",
                "stocks.last_updated",
                "news.sentiments_min_1h",
                "news.sentiments_max_1h",
                "news.sentiment"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [39]:
# nuclio: end-code

In [40]:
# test local
init_context(context)
event = ""#nuclio.Event(body='')
df = handler(context,event)

Python> 2021-03-22 13:53:53,049 [info] Initalizing context & constructing featrure vector
Python> 2021-03-22 13:53:53,088 [info] Reading features from feature vector
Python> 2021-03-22 13:53:53,146 [info] GOOGL
Python> 2021-03-22 13:53:53,167 [info] [[2016.4599609375, 2029.8399658203125, 0.0, 0.0, 2029.839965820312, 0.0, '2021-03-22 09:53:21-04:00', -2.3333333333333335, -2.3333333333333335, -2.333333333333333]]
Python> 2021-03-22 13:53:53,171 [info] MSFT
Python> 2021-03-22 13:53:53,176 [info] [[232.07000732421875, 233.0863037109375, 0.0, 0.0, 233.0863037109375, 0.0, '2021-03-22 09:53:25-04:00', -1.3333333333333333, -1.3333333333333333, -1.333333333333333]]
Python> 2021-03-22 13:53:53,179 [info] AMZN
Python> 2021-03-22 13:53:53,183 [info] [[3078.739990234375, 3096.719970703125, 0.0, 0.0, 3096.719970703125, 0.0, '2021-03-22 09:53:22-04:00', -2.0, -1.6666666666666667, -2.0]]
Python> 2021-03-22 13:53:53,187 [info] AAPL
Python> 2021-03-22 13:53:53,191 [info] [[121.25, 121.45999908447266, 0.

In [ ]:
["prices_min_1h","prices_max_1h","volumes_min_1h","volumes_max_1h","price","volume","last_updated",
"sentimenst_min_1h","sentiments_max_1h","sentiment"]

In [ ]:
df